In [1]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit #for data preprocessing and crass validating 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LogisticRegression #logistic Regression
from sklearn.ensemble import RandomForestRegressor #Random Forest 

from statistics import mean
from hyperopt import Trials, hp, fmin, tpe, STATUS_OK, space_eval #for hyperparameter tuning and minimizing

from cyclic_boosting.pipelines import pipeline_CBClassifier
from sklearn.ensemble import HistGradientBoostingRegressor

from datetime import date
from datetime import datetime

import tensorflow as tf

import keras
import keras.layers as layers
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.losses import MeanSquaredError

import itertools
from keras.layers import LSTM

import seaborn as sns 

from scipy.stats import boxcox 
from scipy.special import inv_boxcox


In [2]:
samples = pd.read_csv('preprocessed_lstm.csv')
samples['date'] = pd.to_datetime(samples['date'])
reading_types = pd.read_csv('reading_types.csv')
# samples.info()

df_lst = [(k, v) for k, v in samples.groupby('building_id')]


In [3]:
from matplotlib import pyplot as plt
from sklearn.discriminant_analysis import StandardScaler
value_type_ids = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']

for building, df in df_lst[0:1]:
    for typeId in value_type_ids:
        train_dates = pd.to_datetime(df['date'])
        multivariate = df.drop(['Unnamed: 0', 'building_id', 'date'], axis = 1)  
        # multivariate.info()
        multivariate = multivariate.astype('float32')
        multivariate = multivariate[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']]

        # boxcox
        # multivariate[typeId] = multivariate[typeId].apply(lambda x: x + 1e-6)
        # df_boxcox_vals, fitted_lambda = boxcox(multivariate[typeId])
        # multivariate[typeId] = df_boxcox_vals

        sz = len(multivariate)
        train_sz = int(sz * 0.5)
        test_sz = len(multivariate) - train_sz

        scaler = StandardScaler()  
        scaler = scaler.fit(multivariate) 
        df_scaled = scaler.transform(multivariate)

        train, test = df_scaled[0:train_sz,:], df_scaled[train_sz:sz,:]

        def create_dataset(dataset, look_back = 1):
            dataX, dataY = [], []
            for i in range(len(dataset) - look_back - 1):
                a = dataset[i:(i + look_back), :]
                dataX.append(a)
                dataY.append(dataset[i + look_back, :])
            return np.array(dataX), np.array(dataY)
        
        look_back = 10
        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        
        # reshape input to be [samples, time steps, features]
        trainX, trainY = np.array(trainX), np.array(trainY)

        print('trainX shape == {}.'.format(trainX.shape))
        print('trainY shape == {}.'.format(trainY.shape)) 

        # x_train, x_test, y_train, y_test = train_test_split(df.drop(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], axis = 1), df[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']])
        # x_train = np.asarray(x_train).astype(np.float32)
        # y_train = np.asarray(y_train).astype(np.float32)
        model = keras.Sequential()
        model.add(layers.LSTM(150,input_shape = (trainX.shape[1], trainX.shape[2]), return_sequences = True))
        model.add(LSTM(50, return_sequences = False))
        # model.add(LSTM(100))
        model.add(layers.BatchNormalization())
        model.add(layers.Dense(12, activation= 'linear'))


        early_stop = EarlyStopping(monitor = 'val_loss', patience = 5)

        model.compile(optimizer= tf.keras.optimizers.Adam(), loss= tf.keras.losses.Huber())

        history = model.fit(trainX, trainY, batch_size=64, validation_split = 0.2, epochs=15, verbose = 1)
        history = model.fit(trainX, trainY, batch_size=64, validation_split = 0.2, epochs=15, verbose = 1, callbacks = [early_stop])
        
        plt.plot(history.history['loss'], label='Training loss')
        plt.plot(history.history['val_loss'], label='Validation loss')
        plt.legend()
        plt.show()

        predict_range = 1
        train_dates = train_dates[0:train_sz]

        n_future = pd.date_range(train_dates.iloc[look_back], periods = predict_range, freq = 'H').tolist()

        prediction = model.predict(trainX[0:predict_range])

        # col = 3

        y_pred_future = scaler.inverse_transform(prediction)[:, 11]
        print(y_pred_future)

        #inverse boxcox
        # y_pred_future = inv_boxcox(y_pred_future, fitted_lambda)
        # y_pred_future = np.vectorize(lambda x: x - 1e-6)(y_pred_future)
        # # y_pred_future = y_pred_future[0] - 1e-6

        forecast_dates = []
        for time_i in n_future:
            forecast_dates.append(time_i.date())


        df_forecast = pd.DataFrame({'date':np.array(forecast_dates), reading_types.at[int(typeId) - 1, 'reading_type_name']:y_pred_future})
        df_forecast['date']=pd.to_datetime(df_forecast['date'])

        original = df[['date', typeId]]
        original['date']=pd.to_datetime(original['date'])
        original = original[look_back:look_back+predict_range]

        print("original", '-'*80)
        print(original.head(1))

        print("df_forecast", '-'*80)
        print(df_forecast.head(1))

        # sns.lineplot(data= original, x = 'date', y = typeId)
        # sns.lineplot(data = df_forecast, x = 'date', y =  reading_types.at[int(typeId) - 1, 'reading_type_name'])
        # plt.show()

trainX shape == (3705, 10, 12).
trainY shape == (3705, 12).
Epoch 1/15
47/47 [==============================] - 7s 47ms/step - loss: 0.2417 - val_loss: 0.3254
Epoch 2/15
47/47 [==============================] - 1s 23ms/step - loss: 0.1623 - val_loss: 0.3353
Epoch 3/15
47/47 [==============================] - 1s 23ms/step - loss: 0.1434 - val_loss: 0.3398
Epoch 4/15
47/47 [==============================] - 1s 24ms/step - loss: 0.1315 - val_loss: 0.3394
Epoch 5/15
47/47 [==============================] - 1s 26ms/step - loss: 0.1212 - val_loss: 0.3293
Epoch 6/15
47/47 [==============================] - 1s 24ms/step - loss: 0.1109 - val_loss: 0.3162
Epoch 7/15
47/47 [==============================] - 1s 25ms/step - loss: 0.1070 - val_loss: 0.2988
Epoch 8/15
47/47 [==============================] - 1s 24ms/step - loss: 0.1036 - val_loss: 0.2765
Epoch 9/15
47/47 [==============================] - 1s 24ms/step - loss: 0.1016 - val_loss: 0.2579
Epoch 10/15
47/47 [==============================

Traceback (most recent call last):
  File "C:\Users\Thomas\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Thomas\AppData\Local\Temp\ipykernel_27924\3478894849.py", line 62, in <module>
    history = model.fit(trainX, trainY, batch_size=64, validation_split = 0.2, epochs=15, verbose = 1, callbacks = [early_stop], warm_start = True)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Thomas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Users\Thomas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 67, in error_handler
    filtered_tb = _process_traceback_frames(e.__tr